# Predicting a company's stock change right after earnings call 
## It is a decision tree model that takes earnings call transcript and earnings surprise and spits out percentage change of stock price and volatility
Modify 2nd cell with company name and url of latest earnings call from seekingalpha.com, then run all cells sequentially.You might also need to add latest earnings surprise to the XX_eps.csv file under data/ as well if it is not updated. 

In [1]:
from data_prep import * 
import pickle

 ...Loading Master Dictionary 85000
Master Dictionary loaded from file: 
  LoughranMcDonald_MasterDictionary_2014.csv
  85,131 words loaded in master_dictionary.



In [2]:
url = "https://seekingalpha.com/article/4237346-amazon-com-inc-amzn-q4-2018-results-earnings-call-transcript"
company_name = 'Amazon'

In [3]:
# Main function for stock prediction
# Inputs: url of latest earnings call and company name
# Outputs: stock price and volatility change 
def post_call_stock_prediction(url, company_name):
    
    # Scrape the earnings call transcript and put it in a dataframe
    time_str, company, title, full_text = get_one_earnings_call(url)
    latest_call = pd.DataFrame(columns = ['time','company','title','full_text','url'])
    latest_call.loc[0] = [time_str,company,title,full_text,url]
    # Convert the time string to Pandas datetime
    # There is about a few hours to half day delay btw earnings call and the time the transcript was posted
    time_earnings = pd.to_datetime(latest_call['time']) - timedelta(days=0.5)
    time_earnings = pd.to_datetime(time_earnings.apply(datetime.date))
    
    # Extract tones from the earnings call transcript
    tones = latest_call['full_text'].apply(parse_one_call_transcript)
    tone_intro = []
    tone_qna = []
    tone_ab = []
    for tone in tones:
        tone_intro.append(tone[0])
        tone_qna.append(tone[1])
        tone_ab.append(tone[2])
    earnings_stock = pd.DataFrame(time_earnings,columns = ['time'])
    ticker = ticker_list[company_name] 
    earnings_stock['company'] = ticker
    earnings_stock['tone_intro'] = tone_intro
    earnings_stock['tone_qna'] = tone_qna
    earnings_stock['tone_ab'] = tone_ab    
    
    # Get the stock data from Yahoo finance and compute the ground-truth changes in the stock
    time_window = [10,10] #ten days before and after call
    stock_change = earnings_stock['time'].apply(compute_stock_prop,args = (ticker,time_window))
    price_change = []
    vola_change = []
    for change in stock_change:
        price_change.append(change[0])
        vola_change.append(change[1])
    
    earnings_stock['price_change'] = price_change
    earnings_stock['vola_change'] = vola_change   

    # Load earnings surprise data and merge with earnings call tones and stock changes
    eps_latest = load_eps(company_name)
    data_latest = eps_latest.merge(earnings_stock,on = 'time')
    print(data_latest)
    X_latest = data_latest[['eps','tone_ab']]
    y_latest = data_latest[['price_change','vola_change']]
    
    # Load pre-trained model
    p = os.path.join(os.getcwd(), "stock_prediction_model.best")
    with open(p, 'rb') as f2:
        best_model = pickle.load(f2)  
    
    # Predict stock price and volatility changes using the model
    y_pred_latest = best_model.predict(X_latest)
    real_price_change = round(y_latest['price_change'][0]*10000)/100
    predicted_price_change = round(y_pred_latest[0,0]*10000)/100
    real_vola_change = round(y_latest['vola_change'][0]*10000)/100
    predicted_vola_change = round(y_pred_latest[0,1]*10000)/100
    print(company_name +' stock price change:' + str(real_price_change) + '%')
    print('predicted price change: ' + str(predicted_price_change) + '%')
    print(company_name +' stock volatility change:' + str(real_vola_change) + '%')
    print('predicted volatility change: ' + str(predicted_vola_change) + '%')
    

In [4]:
post_call_stock_prediction(url, company_name)

        time   eps company  tone_intro  tone_qna  tone_ab  price_change  \
0 2019-01-31  8.83    AMZN   -1.550388  1.097143 -2.64753     -0.021386   

   vola_change  
0     -0.39332  
Amazon stock price change:-2.14%
predicted price change: -2.07%
Amazon stock volatility change:-39.33%
predicted volatility change: 21.51%
